In [1]:
from safetensors.torch import load_file
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from sagemaker.pytorch import PyTorch

2025-02-20 18:38:18.953625: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [19]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from safetensors.torch import load_file
import torch
from sagemaker.serve.mode.function_pointers import Mode
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.serve import InferenceSpec
import boto3
from sagemaker.serve import CustomPayloadTranslator
import torch
from torchvision.transforms import transforms
from sagemaker.serve.spec.inference_spec import InferenceSpec
from transformers import pipeline
from sagemaker.serve import ModelServer
from sagemaker.pytorch import PyTorchModel
import sagemaker
from pathlib import Path
import numpy as np
import io
import os
sagemaker_session = Session()
region = boto3.Session().region_name

# get execution role
# please use execution role if you are using notebook instance or update the role arn if you are using a different role
execution_role = get_execution_role() if get_execution_role() is not None else "your-role-arn"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


2025-02-20 23:43:17.462922: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:matplotlib.font_manager:generated new fontManager


In [13]:
import unsloth

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

## Loading the Finetuned Model
SAVED_MODEL is the lora huggingface repo of the model that has been fine-tuned. INFER_MODEL is the huggingface repo of the model that has the lora weights merged with the base weights for inferring.

In [11]:
SAVED_MODEL = "Alexis-Az/Story-Generation-LlaMA-3.1-8B-10k"
INFER_MODEL= "Alexis-Az/Story-Generation-Model"
max_seq_length = 1024

In [12]:
adapter_model, tokenizer = unsloth.FastLanguageModel.from_pretrained(SAVED_MODEL, load_in_4bit=True)

NameError: name 'unsloth' is not defined

In [9]:
adapter_model.save_pretrained_merged("Story-Generation-LlaMA-3.1-8B-10k", tokenizer)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 15.61 out of 30.89 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 41%|████      | 13/32 [00:01<00:01, 14.00it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:14<00:00,  2.18it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [ ]:
adapter_model

In [9]:
!ls

'Finetuning Story Generation Model.ipynb'   hf_auth.ipynb
 README.md				    inference_container
 Story-Generation-LlaMA-3.1-8B-10k	    load_data.ipynb
 feature_engineering.ipynb		    unsloth_compiled_cache
 format_pth_model.ipynb


## Saving the Merged Model 
The model will be pushed to the INFER_MODEL huggingface repo, and will also be saved as a model .pth file for use in the container for inferrence.

In [2]:
pth_model = AutoModelForCausalLM.from_pretrained("./Story-Generation-LlaMA-3.1-8B-10k")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("./Story-Generation-LlaMA-3.1-8B-10k")

In [6]:
pth_model.push_to_hub(
    INFER_MODEL,
    tokenizer=tokenizer,
    safe_serialization=True,
    create_pr=True,
    max_shard_size="3GB",
)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Alexis-Az/Story-Generation-Model/commit/785a0842eca3e73c6ff7594dc7d1574f78353ce3', commit_message='Upload LlamaForCausalLM', commit_description='', oid='785a0842eca3e73c6ff7594dc7d1574f78353ce3', pr_url='https://huggingface.co/Alexis-Az/Story-Generation-Model/discussions/3', repo_url=RepoUrl('https://huggingface.co/Alexis-Az/Story-Generation-Model', endpoint='https://huggingface.co', repo_type='model', repo_id='Alexis-Az/Story-Generation-Model'), pr_revision='refs/pr/3', pr_num=3)

In [11]:
from pathlib import Path
model_dir = "./serve"
!mkdir -p {model_dir}

In [3]:
torch.save(pth_model.state_dict(), model_dir + '/model.pth')

In [5]:
!ls

'Finetuning Story Generation Model.ipynb'   inference_container
 README.md				    load_data.ipynb
 deploy_model.ipynb			    sample_input.txt
 feature_engineering.ipynb		    unsloth_compiled_cache
 format_pth_model.ipynb			    working_dir
 hf_auth.ipynb


# Using Sagemaker's 'ModelBuilder' Class to Format the PyTorch Model Artifacts

## Inference Object for Handling Input/Output of Model

In [2]:
class InferenceSpec(InferenceSpec):
    def invoke(self, input_object: object, model: object):       
        with torch.no_grad():
            output = model(input_object)
        return output
        
    def load(self, model_dir: str):
        model = AutoModelForCausalLM
        model.load_state_dict(torch.load(model_dir+'/model.pth'))
        model.eval()
        return model

inf_spec = InferenceSpec()

In [32]:
instance_type = 'ml.g5.2xlarge'
image = image_uris.retrieve(region=region, framework='pytorch', image_scope='inference', version='2.0.0', base_framework_version='pytorch2.0.0', instance_type=instance_type)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py310


In [33]:
value: str = "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:"
schema = SchemaBuilder(value,
            {"generated_text": "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\\nDaniel: Hello, Girafatron!\\nGirafatron: Hi, Daniel. I was just thinking about how magnificent giraffes are and how they should be worshiped by all.\\nDaniel: You and I think alike, Girafatron. I think all animals should be worshipped! But I guess that could be a bit impractical...\\nGirafatron: That\'s true. But the giraffe is just such an amazing creature and should always be respected!\\nDaniel: Yes! And the way you go on about giraffes, I could tell you really love them.\\nGirafatron: I\'m obsessed with them, and I\'m glad to hear you noticed!\\nDaniel: I\'"})

In [34]:
model_dir = "./serve"
local_model_dir = str(Path(model_dir).resolve())

In [35]:
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    inference_spec=inf_spec,
    model_path=model_dir,
    role_arn=execution_role,
    image_uri=image,
    schema_builder=schema,
    env_vars={"HF_TASK":"text-generation"},
    model_server=ModelServer.TORCHSERVE
)

In [36]:
model_builder

ModelBuilder(model_path='./working_dir/models/serve', role_arn='arn:aws:iam::597161074694:role/service-role/SageMaker-ExecutionRole-20250210T145384', sagemaker_session=None, name='model-name-0f25e088efd611efa4457e001caa723d', mode=<Mode.SAGEMAKER_ENDPOINT: 3>, shared_libs=[], dependencies={'auto': False}, env_vars={'HF_TASK': 'text-generation'}, log_level=10, content_type=None, accept_type=None, s3_model_data_url=None, instance_type='ml.c5.xlarge', schema_builder=SchemaBuilder(
input_serializer=<sagemaker.base_serializers.StringSerializer object at 0x7f8b93abbc90>
output_serializer=<sagemaker.serve.builder.schema_builder.JSONSerializerWrapper object at 0x7f8b92a57b50>
input_deserializer=<sagemaker.base_deserializers.StringDeserializer object at 0x7f8ba0770950>
output_deserializer=<sagemaker.base_deserializers.JSONDeserializer object at 0x7f8b92a57c10>), model=None, inference_spec=<__main__.InferenceSpec object at 0x7f8ff5c30390>, image_uri='763104351884.dkr.ecr.us-east-1.amazonaws.com/

In [ ]:
aws_model = model_builder.build(role_arn=execution_role, sagemaker_session=sagemaker_session)

In [5]:
source_directory = "./serve"  # Replace with the actual directory path
output_file = "model"

In [17]:
!tar --use-compress-program="pigz --best --recursive" -cf 'model.tar.gz' './serve'

In [20]:
import boto3

# 1. Create an S3 client
s3 = boto3.client('s3', region_name=region)  # Replace 'YOUR_REGION' with your bucket's region

# 2. Define file and bucket details
file_path = './model.tar.gz'   # Path to the local file you want to upload
bucket_name = 'unsloth-finetuned'      # Name of your S3 bucket
object_key = 'model.tar.gz'  # Desired key (filename) for the object in S3

try:
    # 3. Upload the file
    s3.upload_file(file_path, bucket_name, object_key)
    print(f"File '{file_path}' uploaded to S3 bucket '{bucket_name}' as '{object_key}'")
except Exception as e:
    print(f"Error uploading file: {e}")


File './model.tar.gz' uploaded to S3 bucket 'unsloth-finetuned' as 'model.tar.gz'
